<a href="https://colab.research.google.com/github/ThanuMahee12/ayush-vision/blob/util/Util/.AnotationFileEdit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Roboflow Anotation File Edit

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import shutil


In [3]:
root="/content/drive/MyDrive/AyushVision/yolo/dataset"

In [4]:
working_dataset=os.path.join("/content/drive/MyDrive/AyushVision/yolo",'workingdir')

In [5]:
shutil.copytree(root,working_dataset)

'/content/drive/MyDrive/AyushVision/yolo/workingdir'

In [ ]:
dirctories=list(map(lambda item: os.path.join(dataset_root,item,'labels'),['train','test','valid']))

In [ ]:
def modify_label_file(file_path, increasenumber):
    # Read the file content line by line
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Replace the first number in each line with the new number
    updated_lines = []
    for line in lines:
        parts = line.split(maxsplit=1)  # Split only on the first space
        if len(parts) > 1:
            updated_line = f"{int(parts[0])+increasenumber} {parts[1]}"
        else:
            updated_line = f"{int(parts[0])+increasenumber}"
        updated_lines.append(updated_line)

    # Write the updated content back to the file
    with open(file_path, 'w') as file:
        file.writelines(updated_lines)

In [ ]:
def process_directory(directory_path, increasenum):
    # Loop through all files in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):  # Process only .txt files
            file_path = os.path.join(directory_path, filename)
            modify_label_file(file_path, increasenum)
            print(f"Processed file: {filename}")


In [ ]:
for dir in dirctories:
    process_directory(dir,21)

Streaming output truncated to the last 5000 lines.
Processed file: IMG20240426174639_jpg.rf.351aca7c191358d9735e60e6757d8a38.txt
Processed file: IMG20240426174639_jpg.rf.391cbf1cc05c50475553b0e22a08cf4b.txt
Processed file: IMG20240426174639_jpg.rf.c83b12d999fff59595a3e5494ba1043d.txt
Processed file: IMG20240426174641_jpg.rf.7cfc9d73aa10a79cea3cd7db327ca475.txt
Processed file: IMG20240426174641_jpg.rf.80f19b4b895e7cbdf82f3d13e444816d.txt
Processed file: IMG20240426174641_jpg.rf.a7b620f2efed659a0e8af8d854237ba6.txt
Processed file: IMG20240426174651_jpg.rf.7e9d721c16bf3abcf2694d097da09142.txt
Processed file: IMG20240426174651_jpg.rf.dcf4256e1aa9ba51080581f8afcc3d31.txt
Processed file: IMG20240426174651_jpg.rf.dffc02e76c4ffd8d7d6078a78ee6158d.txt
Processed file: IMG20240426174708_jpg.rf.47797ca365c06be55bb6f56a599c04a1.txt
Processed file: IMG20240426174708_jpg.rf.492758656c5f7837bb530b44da5989f5.txt
Processed file: IMG20240426174708_jpg.rf.8c3652aed46017880aa9deae20351146.txt
Processed fil

In [ ]:
import glob
file_paths = glob.glob(f"{dataset_root}/**/labels/*.txt", recursive=True)
len(file_paths)

13191

In [ ]:
import glob
import re

# Search for all txt files in labels subfolders
file_paths = glob.glob(f"{dataset_root}/**/labels/*.txt", recursive=True)
classnames={}
for file_path in file_paths:
    with open(file_path, 'r', encoding='utf-8') as f:
        classes = list(map(lambda line:line.split(' ')[0],f.readlines()))
        for className in list(set(classes)):
          classnames[className]=classnames.get(className,0)+1


In [ ]:
classnames

{'21': 1200,
 '25': 1200,
 '22': 1199,
 '24': 1199,
 '26': 1200,
 '23': 1200,
 '31': 1199,
 '27': 1199,
 '29': 1200,
 '30': 1198,
 '28': 1197}

In [ ]:
shutil.copytree("/content/drive/MyDrive/AyushVision/yolo/workingdir/ayushvision_plants-2lgbz","/content/drive/MyDrive/AyushVision/yolo/final")

'/content/drive/MyDrive/AyushVision/yolo8/dataset/final'

In [6]:
import yaml
import os

# Load YAML files and extract class names
def load_yaml(file_path):
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)
    return data['names']

# Generate a mapping of old class IDs to new class IDs
def generate_class_mapping(data1_classes, data2_classes):
    class_mapping = {}
    for old_class_id, class_name in enumerate(data1_classes):
        if class_name in data2_classes:
            new_class_id = data2_classes.index(class_name)
            class_mapping[old_class_id] = new_class_id
    return class_mapping



In [11]:
# Update annotation files with new class IDs
def update_annotation_file(file_path, class_mapping):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    updated_lines = []
    for line in lines:
        parts = line.strip().split()
        old_class_id = int(parts[0])

        # Remap the class ID using the class mapping
        if old_class_id in class_mapping:
            new_class_id = class_mapping[old_class_id]
            updated_line = f"{new_class_id} " + " ".join(parts[1:]) + "\n"
            updated_lines.append(updated_line)
        else:
            updated_lines.append(line)  # If no mapping, keep original

    # Write back the updated annotation file
    with open(file_path, 'w') as file:
        file.writelines(updated_lines)

# Update all annotation files in the directory
def update_all_annotations(data_dir, class_mapping):
  dirs=list(map(lambda x:os.path.join(data_dir,x,'labels'),['train','valid','test']))
  count=0
  for annotation_dir in dirs:
    for filename in os.listdir(annotation_dir):
        if filename.endswith('.txt'):  # Only process annotation files
            file_path = os.path.join(annotation_dir, filename)
            update_annotation_file(file_path, class_mapping)
            print(f"Updated {file_path}")
            count+=1
  print(f"Updated {count} files")




In [20]:
# Paths to data1.yaml and data2.yaml
data1_yaml_path = "/content/drive/MyDrive/AyushVision/yolo/dataset/ayushvision_plant/data.yaml"
data2_yaml_path = "/content/drive/MyDrive/AyushVision/yolo/workingdir/ayushvision_plant/data.yaml"


# Load class names from data1.yaml and data2.yaml
data1_classes = load_yaml(data1_yaml_path)
data2_classes = load_yaml(data2_yaml_path)

# Generate class mapping from data1 to data2
class_mapping = generate_class_mapping(data1_classes, data2_classes)

# Print the class mapping for verification
print("Class Mapping (data1 -> data2):")
for old_id, new_id in class_mapping.items():
    print(f"{old_id} ({data1_classes[old_id]}) -> {new_id} ({data2_classes[new_id]})")


Class Mapping (data1 -> data2):
0 (Acalyphaindica) -> 21 (Acalyphaindica)
1 (Adamant_Creeper) -> 3 (Adamant_Creeper)
2 (Alovera) -> 4 (Alovera)
3 (Betel) -> 5 (Betel)
4 (Black_honey_shrub) -> 6 (Black_honey_shrub)
5 (Calotropis_gigantea) -> 7 (Calotropis_gigantea)
6 (Caster) -> 8 (Caster)
7 (Cinnamon) -> 9 (Cinnamon)
8 (Ginger) -> 10 (Ginger)
9 (Henna) -> 11 (Henna)
10 (Hibiscus) -> 0 (Hibiscus)
11 (Holybasil) -> 12 (Holybasil)
12 (Indian_Mallow) -> 13 (Indian_Mallow)
13 (Ivy_gourd) -> 14 (Ivy_gourd)
14 (Landrina) -> 15 (Landrina)
15 (Lemon_grass) -> 16 (Lemon_grass)
16 (Miracle_leaf) -> 18 (Miracle_leaf)
17 (Mountain_Knotgrass) -> 19 (Mountain_Knotgrass)
18 (Neem) -> 20 (Neem)
19 (Oregano) -> 22 (Oregano)
20 (Pedaliummurex) -> 23 (Pedaliummurex)
21 (Redpea_egg_plant) -> 24 (Redpea_egg_plant)
22 (Ringworm_plant) -> 25 (Ringworm_plant)
23 (Seedsunder_leaf) -> 26 (Seedsunder_leaf)
24 (Sleepy_plant) -> 1 (Sleepy_plant)
25 (Tallperennial_grass) -> 27 (Tallperennial_grass)
26 (Thornapple) -

In [21]:
annotation_dir = "/content/drive/MyDrive/AyushVision/yolo/workingdir/ayushvision_plant"

In [22]:

# Update all annotation files in the directory
update_all_annotations(annotation_dir, class_mapping)

Streaming output truncated to the last 5000 lines.
Updated /content/drive/MyDrive/AyushVision/yolo/workingdir/ayushvision_plant/train/labels/IMG20240425090935_jpg.rf.d1e64ac62912d3dd1d5f63a728e860df.txt
Updated /content/drive/MyDrive/AyushVision/yolo/workingdir/ayushvision_plant/train/labels/IMG20240425090937_jpg.rf.a8c1b76cdf09b7c1e7812b219032d711.txt
Updated /content/drive/MyDrive/AyushVision/yolo/workingdir/ayushvision_plant/train/labels/IMG20240425090939_jpg.rf.019cb088de0479dd4a56438a20af404d.txt
Updated /content/drive/MyDrive/AyushVision/yolo/workingdir/ayushvision_plant/train/labels/IMG20240425090940_jpg.rf.e5d331bb96d62037c1a23984aef7a66e.txt
Updated /content/drive/MyDrive/AyushVision/yolo/workingdir/ayushvision_plant/train/labels/IMG20240425090942_jpg.rf.b1a85e570fe9f113245df5fa4df75583.txt
Updated /content/drive/MyDrive/AyushVision/yolo/workingdir/ayushvision_plant/train/labels/IMG20240425090943_jpg.rf.2e9c4d980c6e3e6ef80d20a9167fa893.txt
Updated /content/drive/MyDrive/AyushV

In [23]:
newpath="/content/drive/MyDrive/AyushVision/yolo/final"

In [27]:
for dirs in os.listdir("/content/drive/MyDrive/AyushVision/yolo/workingdir"):
    for splipath in os.listdir(os.path.join("/content/drive/MyDrive/AyushVision/yolo/workingdir",dirs)):
      if os.path.isdir(os.path.join("/content/drive/MyDrive/AyushVision/yolo/workingdir",dirs,splipath)):
        for anotatedirs in os.listdir(os.path.join("/content/drive/MyDrive/AyushVision/yolo/workingdir",dirs,splipath)):
          files=os.listdir(os.path.join("/content/drive/MyDrive/AyushVision/yolo/workingdir",dirs,splipath,anotatedirs))
          filepaths=list(map(lambda x:os.path.join("/content/drive/MyDrive/AyushVision/yolo/workingdir",dirs,splipath,anotatedirs,x),files))
          newdir=os.path.join(newpath,splipath,anotatedirs)
          if not os.path.exists(newdir):
            os.makedirs(newdir,exist_ok=True)
          for filepath in filepaths:
            shutil.copy(filepath,newdir)
          print(f"Copied {len(filepaths)} files to{dirs}/{splipath}/{anotatedirs}--> {newdir}")


Copied 60 files toayushvision_plants-2lgbz/test/images--> /content/drive/MyDrive/AyushVision/yolo/final/test/images
Copied 60 files toayushvision_plants-2lgbz/test/labels--> /content/drive/MyDrive/AyushVision/yolo/final/test/labels
Copied 420 files toayushvision_plants-2lgbz/train/images--> /content/drive/MyDrive/AyushVision/yolo/final/train/images
Copied 420 files toayushvision_plants-2lgbz/train/labels--> /content/drive/MyDrive/AyushVision/yolo/final/train/labels
Copied 120 files toayushvision_plants-2lgbz/valid/images--> /content/drive/MyDrive/AyushVision/yolo/final/valid/images
Copied 120 files toayushvision_plants-2lgbz/valid/labels--> /content/drive/MyDrive/AyushVision/yolo/final/valid/labels
Copied 894 files toayushvision_plants/test/images--> /content/drive/MyDrive/AyushVision/yolo/final/test/images
Copied 894 files toayushvision_plants/test/labels--> /content/drive/MyDrive/AyushVision/yolo/final/test/labels
Copied 6258 files toayushvision_plants/train/images--> /content/drive/